In [1]:
import tensorflow as tf
import numpy as np
import model
import util
import dataset
import time
import os
import progressbar

### Setting params, dirs, and loading data

In [2]:
paths = ['./out/', './params']
for path in paths:
    if not os.path.exists(path):
        os.makedirs(path)
                
data = dataset.read()
batch_size = 100
learning_rate = 0.0001
beta1 = 0.5
z_size = 51200
save_interval = 1



###  Creating Placeholders

In [3]:
x = tf.placeholder(tf.float32, [batch_size, 32, 32, 32, 1])
z = tf.placeholder(tf.float32, [batch_size, z_size])
train = tf.placeholder(tf.bool)

### Create Generator and Discriminator Object
#### Setting up labels, loss functions, other parameters

In [4]:

G = model.Generator(z_size)
D = model.Discriminator()
x_ = G(z)
y_ = D(x_, train)

y = D(x, train)

label_real = np.zeros([batch_size, 2], dtype=np.float32)
label_fake = np.zeros([batch_size, 2], dtype=np.float32)
label_real[:, 0] = 1
label_fake[:, 1] = 1

loss_G = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_, tf.constant(label_real)))
loss_D = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_, tf.constant(label_fake)))
loss_D += tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, tf.constant(label_real)))

var_G = [v for v in tf.trainable_variables() if 'g_' in v.name]
var_D = [v for v in tf.trainable_variables() if 'd_' in v.name]

opt_G = tf.train.AdamOptimizer(learning_rate, beta1).minimize(loss_G, var_list=var_G)
opt_D = tf.train.AdamOptimizer(learning_rate, beta1).minimize(loss_D, var_list=var_D)

saver = tf.train.Saver()

### Setting up Progressbar

In [5]:
progress = progressbar.ProgressBar()

### Running Voxel-dcgan

In [6]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:
    
    #init = 
    sess.run(tf.initialize_all_variables(), feed_dict={train:True})
    total_batch = data.train.num_examples / batch_size

    for epoch in progress(xrange(1, 125)):  # Repeat this process 50 times

        for i in xrange(total_batch):
            voxels = np.expand_dims(data.train.next_batch(batch_size), 4)
            batch_z = np.random.uniform(-1, 1, [batch_size, z_size]).astype(np.float32)

            sess.run(opt_G, feed_dict={z:batch_z, train:True})
            sess.run(opt_D, feed_dict={x:voxels, z:batch_z, train:True})

            with open("out/loss.csv", 'a+') as f:
                batch_loss_G = sess.run(loss_G, feed_dict={z:batch_z, train:False})
                batch_loss_D = sess.run(loss_D, feed_dict={x:voxels, z:batch_z, train:False})
                msg = "{0}, {1}, {2:.8f}, {3:.8f}".format(epoch, i, batch_loss_G, batch_loss_D)
                print f.write(msg+'\n')
                print msg

        batch_z = np.random.uniform(-1, 1, [batch_size, z_size]).astype(np.float32)
        voxels = sess.run(x_, feed_dict={z:batch_z})

        for j, v in enumerate(voxels[:5]):
            v = v.reshape([32, 32, 32]) > 0

            util.save_binvox(v, "out/epoch{0}-{1}.binvox".format(epoch, j))

        if epoch % save_interval == 0:
            saver.save(sess, "params/epoch{0}.ckpt".format(epoch))


Instructions for updating:
Use `tf.global_variables_initializer` instead.


None
1, 0, 1.26538122, 1.55777740
None
1, 1, 1.27999210, 1.06210113
None
2, 0, 1.37645137, 0.88225579
None
2, 1, 1.33455348, 0.69189262


None
3, 0, 1.60014629, 0.62097591
None
3, 1, 1.79811549, 0.50311804
None
4, 0, 1.92418027, 0.46073648
None
4, 1, 2.11651897, 0.36035502


None
5, 0, 2.29039168, 0.32107612
None
5, 1, 2.52181745, 0.29001969
None
6, 0, 2.74664330, 0.25642315
None
6, 1, 2.72030330, 0.20314735


None
7, 0, 2.86813831, 0.17999414
None
7, 1, 2.78494954, 0.18851417
None
8, 0, 2.94692087, 0.19495645
None
8, 1, 3.04301453, 0.16738665


...

None
11, 0, 3.41408992, 0.13045952
None
11, 1, 3.24900150, 0.15139250
None
12, 0, 3.32897758, 0.14054294
None
12, 1, 3.48774791, 0.10990675


None
13, 0, 3.56914425, 0.11671151
None
13, 1, 3.56296086, 0.12867184
None
14, 0, 3.62463140, 0.10098186
None
14, 1, 3.70536947, 0.12575082


None
15, 0, 3.88139534, 0.12242895
None
15, 1, 3.70476246, 0.11225801
None
16, 0, 3.67084694, 0.07993546
None
16, 1, 3.93777251, 0.11376132


None
17, 0, 3.98676538, 0.08344387


KeyboardInterrupt: 

In [ ]:
#